In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from import_train_test import Import_path_to_data
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_path, test_path = Import_path_to_data.Import_train_test()

In [3]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [4]:
def prep_data(data_path, is_train):
    batch_size = 32
    data = tf.keras.utils.image_dataset_from_directory(data_path, batch_size=batch_size, image_size=(100, 100), shuffle=True)
    data.map(lambda x, y: (x / 255, y))
    data.as_numpy_iterator().next()
    if is_train == 1:
        val_size = int(len(data)*.2)
        train_size = int(len(data)*.8)
        val_data = data.take(val_size)
        train_data = data.skip(val_size).take(train_size)
        return train_data, val_data
    else:
        return data

In [5]:
train_data, val_data = prep_data(train_path, 1)
test_data = prep_data(test_path, 0)

Found 67692 files belonging to 131 classes.
Found 22688 files belonging to 131 classes.


In [6]:
from keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Dropout, Input
from keras import Sequential
from keras.activations import relu, softmax
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from functools import partial
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [8]:
model = Sequential([
    Input((100, 100, 3)),
    Conv2D(64, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(64, (3, 3), 1, activation=relu, padding='same'),
    MaxPool2D((2, 2)),
    Conv2D(128, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(128, (3, 3), 1, activation=relu, padding='same'),
    MaxPool2D((2, 2)),
    Conv2D(256, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(256, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(256, (3, 3), 1, activation=relu, padding='same'),
    MaxPool2D((2, 2)),
    Conv2D(512, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(512, (3, 3), 1, activation=relu, padding='same'),
    Conv2D(512, (3, 3), 1, activation=relu, padding='same'),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(units=4096, activation=relu),
    Dense(units=4096, activation=relu),
    Dense(units=131, activation=softmax)
])

In [9]:
es = EarlyStopping(
    monitor="val_loss",
    patience=2,
    mode=min,
    restore_best_weights=True
)

mc = ModelCheckpoint(
    'best_model',
    monitor="val_loss",
    mode=max,
    )

In [10]:
model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])
history = model.fit(train_data, epochs=10,
                    validation_data=(val_data),
                    callbacks=[es, mc])
score = model.evaluate(test_data)

Epoch 1/10
1692/1692 [==============================] - ETA: 0s - loss: 0.4657 - accuracy: 0.8797

INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


1692/1692 [==============================] - 133s 76ms/step - loss: 0.4657 - accuracy: 0.8797 - val_loss: 0.1626 - val_accuracy: 0.9484
Epoch 2/10
1692/1692 [==============================] - ETA: 0s - loss: 0.0674 - accuracy: 0.9815

INFO:tensorflow:Assets written to: best_model\assets


...

INFO:tensorflow:Assets written to: best_model\assets


1692/1692 [==============================] - 131s 76ms/step - loss: 0.0403 - accuracy: 0.9897 - val_loss: 0.1544 - val_accuracy: 0.9741
Epoch 5/10
1692/1692 [==============================] - ETA: 0s - loss: 0.0213 - accuracy: 0.9950

INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


709/709 [==============================] - 15s 21ms/step - loss: 0.1179 - accuracy: 0.9740
